# **🏠 부동산 실거래가 예측 모델링**

> 본 노트는 파이프라인과 러너를 구성해 실험을 재현하는 과정을 정리한 문서입니다.  
> 실제 코드 실행 흐름을 먼저 보여주고, 이후 데이터셋 빌더와 각 파이프라인 블록의 역할을 수도코드 형태로 설명합니다.

## Contents
- Overview
- Library Import
- Data Load
- Data Preprocessing
- Feature Engineering
- Model Training
- Inference
- Output File Save

## 1. Overview
- 실험 재현성과 실수 방지를 위해 **데이터셋 빌더 + 러너 구조**로 구성했습니다.
- 전처리/피처 엔지니어링을 한 곳에서 관리해 실험 간 일관성을 확보했습니다.
- 실행 로그와 결과 파일을 자동으로 저장해 실험 추적이 가능하도록 했습니다.

## 2. Library Import

In [1]:
import numpy as np
import pandas as pd

from lab_core.hyj.hpp_dataset_builder import HPPDatasetBuilder
from lab_core.hyj.core.rf_lgbm_runner import ModelParams, RfLgbmRunner

## 3. Data Load

- 대회 기본 제공 데이터셋
  - 서울 아파트 거래 데이터 (학습용): 실거래가 포함
  - 서울 아파트 거래 데이터 (테스트용): 실거래가 미포함(예측 대상)
  - 지하철/버스 데이터: 역·정류장 좌표 및 노선 정보
- 외부 데이터셋
  - k-apt 데이터: 서울시 공공주택 아파트 정보, 단지 정보/좌표 보정/규모 피처
  - coord 데이터: Kakao API 수집 아파트 위경도 정보, 좌표 결측 보정

데이터 로드는 빌더 내부에서 관리합니다. 실행부에서는 빌더를 생성하고, 러너가 내부적으로 데이터를 불러옵니다.
> 관련 코드: [hpp_dataset_builder.py](../../src/lab_core/hyj/hpp_dataset_builder.py) (load_raw/load_ext)

In [ ]:
load_raw():
  train, test, subway, bus = read_csv(...)
  return train, test, target_col

load_ext():
  apt, coord = read_csv(...)
  return {subway, bus, apt, coord}

In [9]:
train = pd.read_csv(raw_data_dir("train.csv"), low_memory=False)
test = pd.read_csv(raw_data_dir("test.csv"), low_memory=False)
subway = pd.read_csv(raw_data_dir("subway_feature.csv"), low_memory=False)
bus = pd.read_csv(raw_data_dir("bus_feature.csv"), low_memory=False)

apt = pd.read_csv(ext_data_dir("k-apt.csv"), low_memory=False)
coord = pd.read_csv(ext_data_dir("coord.csv"), low_memory=False)

`low_memory=False`는 CSV를 청크로 나눠 읽을 때 생길 수 있는 dtype 혼합 문제를 피하기 위한 옵션입니다.  
전체를 한 번에 읽어 dtype 추론을 안정화하지만, 메모리 사용량은 늘어납니다.

In [10]:
# 로드된 데이터들을 살펴봅니다.
print(train.shape)
display(train.head(1))
print(test.shape)
display(test.head(1))
print(subway.shape)
display(subway.head(1))
print(bus.shape)
display(bus.head(1))
print(apt.shape)
display(apt.head(1))
print(coord.shape)
display(coord.head(1))

(1118822, 52)


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


(9272, 51)


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,...,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0


(768, 5)


...

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로


(2852, 47)


,번호,k-아파트코드,k-아파트명,"k-단지분류(아파트,주상복합등등)",kapt도로명주소,주소(시도)k-apt주소split,주소(시군구),주소(읍면동),나머지주소,주소(도로명),...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,1,A15679103,우리유앤미,아파트,서울특별시 동작구 서달로 83,서울,동작구,흑석동,우리유앤미아파트,서달로,...,위탁,1773.56,223.0,의무,59:42.0,Y,N,126.959639,37.500668,46:59.0


(8477, 3)


,매핑키,좌표X,좌표Y
0,강동구 아리수로50길 38,NaN,NaN


## 4. Data Preprocessing

### 4.1 컬럼명 정리

전처리는 컬럼명 정리부터 수행했습니다.
모든 컬럼을 변경하지 않고, 학습에 사용할 피처 후보군만 대상으로 이름을 정리했습니다.

초기에는 컬럼명을 거의 수정하지 않고 진행했으나, 단위 표기 등 특수 문자가 포함된 컬럼명으로 인해 오류가 발생하여 컬럼명부터 정돈하기로 했습니다.

In [11]:
rename_map = {
    "전용면적(㎡)": "전용면적",
    "k-건설사(시공사)": "건설시공사",
    "k-단지분류(아파트,주상복합등등)": "단지분류",
    "k-세대타입(분양형태)": "세대분양형태",
    "k-관리방식": "관리방식",
    "k-복도유형": "복도유형",
    "k-난방방식": "난방방식",
    "k-전체동수": "전체동수",
    "k-전체세대수": "전체세대수",
    "k-시행사": "시행사",
    "k-연면적": "연면적",
    "k-주거전용면적": "주거전용면적",
    "k-관리비부과면적": "관리비부과면적",
    "k-전용면적별세대현황(60㎡이하)": "전용면적세대수_60",
    "k-전용면적별세대현황(60㎡~85㎡이하)": "전용면적세대수_60_85",
    "k-85㎡~135㎡이하": "전용면적세대수_85_135",
    "k-135㎡초과": "전용면적세대수_135",
    "세대전기계약방법": "세대전기계약방법",
    "청소비관리형태": "청소비관리형태",
    "건축면적": "건축면적",
    "주차대수": "주차대수",
    "기타/의무/임대/임의=1/2/3/4": "의무_임대_임의_기타",
}

train = train.rename(columns=rename_map)
test = test.rename(columns=rename_map)

### 4.2 타깃 분리 및 인덱스 정합 처리

베이스라인처럼 train/test를 합치지 않고, 학습용 target을 먼저 분리해 관리했습니다.
전처리 과정에서 행 소실이나 정렬 변경이 발생할 수 있으므로,
원본 인덱스를 보존하는 `_org_idx` 컬럼을 추가해 X_train과 y_train을 다시 합칠 때 문제가 되지 않도록 했습니다.

In [12]:
target_col = "target"

y_train = train[target_col].copy()
X_train = train.drop(columns=[target_col]).copy()
X_test = test.copy()

X_train["_org_idx"] = X_train.index
X_test["_org_idx"] = X_test.index

### 4.3 사전 분할(데이터 누수 방지)

평가에 사용할 검증 데이터를 전처리 이전에 분리하여, 전처리 과정에서의 데이터 누수를 원천적으로 차단했습니다.  
특히 시간 순서가 중요한 문제이므로, 시간 기반 홀드아웃으로 미래 시점을 검증 세트로 분리하는 방식이 타당하다고 판단했습니다.  
검증 구간은 테스트 데이터셋의 최근 구간과 유사한 시기를 맞추기 위해 2023-04 ~ 2023-06으로 설정했습니다.
이는 무작위 분할보다 실제 예측 환경과 유사한 평가를 만들고 데이터 누수를 줄이기 위한 선택입니다.

In [15]:
# split (전처리 전에 분할)
ym = pd.to_numeric(X_train["계약년월"], errors="coerce")
train_mask = ym <= 202303
valid_mask = (ym >= 202304) & (ym <= 202306)

X_train_base = X_train.loc[train_mask].copy()
y_train_base = y_train.loc[train_mask].copy()
X_valid = X_train.loc[valid_mask].copy()
y_valid = y_train.loc[valid_mask].copy()

### 4.4 무의미 값 처리

`거래유형` 컬럼에 있는 `-` 값은 의미 없는 값으로 판단하여 결측 처리했습니다.
`등기신청일자`와 `중개사소재지` 등에서도 의미 없는 값이 확인되었지만, 실거래가와 상관관계가 없다고 판단되어 변수로 선택하지 않기로 했습니다.

In [ ]:
X_train_base["거래유형"] = X_train_base["거래유형"].replace({"-": np.nan})
X_valid["거래유형"] = X_valid["거래유형"].replace({"-": np.nan})
X_test["거래유형"] = X_test["거래유형"].replace({"-": np.nan})